https://nilearn.github.io/stable/auto_examples/04_glm_first_level/plot_localizer_surface_analysis.html
Volumetric to surface space pipeline for participant 2.
- (V) import our volumetric dataset for participant 2 from  sm-prf, derivatives, fmriprep folder
   test if it actually works w cluster
- check TR
- import our paradigm (Can this just be empty for rs?)
- is the fsaverage5 mesh template ok or do we use an individual mesh? Default template corresponds to MNI
- check if nodes correspond for both meshes in vol_to_surf, and then if this matches the fMRI image. If this is not the case, check 'kind' in vol_to_surf. Everything needs to be in MNI space? Or at least in the same participant space, check what the cluster data look like.
- vol_to_surf interpolation, linear or nearest neighbor?
- low priority: check how segmentation works bc I think we need to do this too no? Bc this code is for both full hemispheres

In [12]:
import nilearn
import numpy as np
import pandas as pd
import nibabel as nib

In [13]:
# prepare timing parameters
t_r = 2.4
slice_time_ref = 0.5 #this means slices are acquired at the middle of the TR

In [14]:
# prepare data
data_dir = "/data1/projects/dumoulinlab/Lab_members/Marijke/SM-pRF/derivatives/fmriprep/sub-002"

# Construct the full path to the fmri image
fmri_img = os.path.join(data_dir, "") #idk which image this needs to be but it needs to be a .nii or .nii.gz image

In [4]:
# import experimental paradigm from example
#import pandas as pd

#events_file = data.events
#events = pd.read_table(events_file)

In [29]:
# downloads a standard fs geometry template. Function takes mesh file (default fsaverage5) + optional data path (empty here). 
fsaverage = nilearn.datasets.fetch_surf_fsaverage()

*vol_to_surf extracts surface data from nifti image*
takes arguments:
- img: earlier defined fmri_img)
- surf_mesh: mesh in .gii format or freesurfer specific files. We use a .gii from the cluster, the pial ones. Do both for lh and rh. Nodes must correspond to nodes in inner_mesh, and then this must correspond to the provided fmri img.)
- radius: ignored bc inner_mesh is provided
- interpolation (?): how vertex values are estimated based on nearby voxel values (not me only learning now that vertices are the points of a triangle, not the triangle itself HAHAHAH ohno). 'linear' by default. Tomas used nearest-neighbor interpolation (though this might be about segmentation?). Linear is slower but more accurate? Still unsure why you'd use either. 
- kind='auto' (?). Automatically becomes 'depth' now that inner_mesh is provided. Just means it picks a sample between two corresponding nodes of the meshes. Not sure what the 'ball' alternative is tho
- n_samples=None by default but the higher the resolution of the data, the higher the nr should be. 10 or 20 seems to be reasonable?
- mask_img=None by default. To specify a ROI so we don't analyze the whole brain
- inner_mesh= white matter mesh in .gii format
- depth = 0.5 because we want sample points to be in between the two meshes

In [ ]:
# check if the nodes of the meshes correspond 
# Load the GIFTI files
pial_left_mesh = nib.load(os.path.join(data_dir, "ses-0/anat/sub-002_ses-0_acq-MP2RAGE_hemi-L_pial.surf.gii"))
wm_left_mesh = nib.load(os.path.join(data_dir, "ses-0/anat/sub-002_ses-0_acq-MP2RAGE_hemi-L_smoothwm.surf.gii"))
pial_right_mesh = nib.load(os.path.join(data_dir, "ses-0/anat/sub-002_ses-0_acq-MP2RAGE_hemi-R_pial.surf.gii"))
wm_right_mesh = nib.load(os.path.join(data_dir, "ses-0/anat/sub-002_ses-0_acq-MP2RAGE_hemi-R_smoothwm.surf.gii"))

# Extract the vertex coordinates from the GIFTI files
vertices_pial_left = pial_left_mesh.darrays[0].data  # Assuming the vertex coordinates are in the first data array
vertices_wm_left = wm_left_mesh.darrays[0].data  # Assuming the vertex coordinates are in the first data array
vertices_pial_right = pial_right_mesh.darrays[0].data  # Assuming the vertex coordinates are in the first data array
vertices_wm_right = wm_right_mesh.darrays[0].data  # Assuming the vertex coordinates are in the first data array

# Check if the vertex coordinates match closely for both hemispheres
if np.allclose(vertices_pial_left, vertices_wm_left, atol=1e-5) and np.allclose(vertices_pial_right, vertices_wm_right, atol=1e-5):
    print("Vertices in both hemisphere meshes correspond closely.")
else:
    print("Vertices in one or both hemisphere meshes do not match.")

In [6]:
# project fmri data to the surface mesh. Texture variable stores the result of the projection
# this consists of the earlier specified fmri image + the rh pial surface mesh from the fsaverage5 template. 
from nilearn import surface

texture_left = surface.vol_to_surf(fmri_img, pial_left, depth = 0.5)

In [16]:
import nibabel as nib
nib.load(fmri_img).get_fdata().shape

(53, 63, 46, 128)

In [11]:
texture.shape

(10242, 128)